<a href="https://colab.research.google.com/github/ABD-01/MNIST/blob/main/MNIST_Using_Multi_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import idx2numpy
from tqdm import tqdm
import os

In [ ]:
test_images = idx2numpy.convert_from_file('/content/drive/MyDrive/Colab Notebooks/t10k-images.idx3-ubyte')
test_labels = idx2numpy.convert_from_file('/content/drive/MyDrive/Colab Notebooks/t10k-labels.idx1-ubyte')
train_images = idx2numpy.convert_from_file('/content/drive/MyDrive/Colab Notebooks/train-images.idx3-ubyte')
train_labels = idx2numpy.convert_from_file('/content/drive/MyDrive/Colab Notebooks/train-labels.idx1-ubyte')

In [ ]:
print(test_images.shape)
print(test_labels.shape)
print(train_images.shape)
print(train_labels.shape)

(10000, 28, 28)
(10000,)
(60000, 28, 28)
(60000,)


In [ ]:
train_images_flat=train_images.reshape(train_images.shape[0], -1).T
test_images_flat=test_images.reshape(test_images.shape[0], -1).T

In [ ]:
m = train_images_flat.shape[1]
m_test = test_images_flat.shape[1]
print(f"Train Examples(m)={m}")
print(f"Test Examples(m_test)={m_test}")
X = train_images_flat/255
X_test = test_images_flat/255
print('X.shape=',X.shape)
print('X_test.shape=',X_test.shape)

Train Examples(m)=60000
Test Examples(m_test)=10000
X.shape= (784, 60000)
X_test.shape= (784, 10000)


In [ ]:
def one_hot(row_vector, no_classes):
    one_hot_vector = np.eye(no_classes)[row_vector.reshape(-1)]
    return one_hot_vector.T

In [ ]:
C = train_labels.max()+1
print(f"No. of Classes={C}")
Y_hot = one_hot(train_labels,no_classes=C)
Y_test_hot = one_hot(test_labels,no_classes=C)
print('Y_hot.shape=',Y_hot.shape) #Y_hot.shape is (C,m)
print('Y_test_hot.shape=',Y_test_hot.shape) #Y_hot.shape is (C,m)

No. of Classes=10
Y_hot.shape= (10, 60000)
Y_test_hot.shape= (10, 10000)


In [32]:
no_of_layers = 4
nodes = [X.shape[0], 512, 128, 32, 10]   # 4 layers with size 512, 128, 32, 10
nodes

[784, 512, 128, 32, 10]

In [36]:
def initialize():
  global WW, bb
  WW = {}
  bb = {}
  for l in range(1,no_of_layers+1):
    WW['W'+str(l)] = np.random.randn(nodes[l], nodes[l-1])*np.sqrt(2/nodes[l-1])
    bb['b'+str(l)] = np.zeros((nodes[l],1))

In [34]:
list(range(1,no_of_layers+1))

[1, 2, 3, 4]

In [76]:
initialize()

In [77]:
WW['W1'].shape, WW['W2'].shape,WW['W3'].shape, WW['W4'].shape

((512, 784), (128, 512), (32, 128), (10, 32))

In [78]:
bb['b1'].shape, bb['b2'].shape,bb['b3'].shape, bb['b4'].shape

((512, 1), (128, 1), (32, 1), (10, 1))

In [80]:
def relu(z):
  a = np.maximum(0,z)
  return a

def softmax(z):
  t = np.exp(z)
  a = t / np.sum(t, keepdims=True, axis=0)
  return a

# def derivative_relu(z)

In [81]:
def forward():
  global ZZ, AA
  ZZ={}
  AA={}
  AA['A'+str(0)] = X
  for l in range(1,no_of_layers):
    ZZ['Z'+str(l)] = np.dot(WW['W'+str(l)], AA['A'+str(l-1)]) +  bb['b'+str(l)]
    AA['A'+str(l)] = relu(ZZ['Z'+str(l)])

  ZZ['Z'+str(no_of_layers)] = np.dot(WW['W'+str(no_of_layers)], AA['A'+str(no_of_layers-1)]) +  bb['b'+str(no_of_layers)]
  AA['A'+str(no_of_layers)] = relu(ZZ['Z'+str(no_of_layers)])

In [82]:
forward()

In [83]:
[(z[0],z[1].shape) for z in ZZ.items()], [(a[0],a[1].shape) for a in AA.items()]

([('Z1', (512, 60000)),
  ('Z2', (128, 60000)),
  ('Z3', (32, 60000)),
  ('Z4', (10, 60000))],
 [('A0', (784, 60000)),
  ('A1', (512, 60000)),
  ('A2', (128, 60000)),
  ('A3', (32, 60000)),
  ('A4', (10, 60000))])

In [94]:
def cost():
  global L,J
  L = -np.sum(Y_hot*np.log(AA['A'+str(no_of_layers)] + 1e-08), keepdims=True, axis=0)
  J = np.sum(L, keepdims=True, axis=1)*(1/m)

In [95]:
cost()

In [70]:
L.shape , J.shape

((1, 60000), (1, 1))

In [72]:
def backward():
  global dZZ, dWW, dbb
  dZZ['dZ'+str(no_of_layers)] = 

In [93]:
AA['A4'].any()

True